In [ ]:
import os
import sys
import numpy as np
import datetime
TIDEH_path = os.path.join('TiDeH-master','TiDeH-master')
sys.path.append(TIDEH_path)

In [ ]:
# Module import
from tideh import estimate_parameters
from tideh import load_events
from tideh import predict
filename = os.path.join(TIDEH_path, 'data/example/sample_file.txt')
(_, start_time), events = load_events(filename)

In [ ]:
# additional parameters passed to infectious rate function
add_params = {'t0': start_time, 'bounds': [(-1, 0.5), (1, 20.)]}
obs_time = 48 # observation time of 2 days
pred_time = 168 # predict for one week
params, err, _ = estimate_parameters(events=events, obs_time=obs_time, **add_params)

In [ ]:
print("Estimated parameters are:")
print("p0:   %.3f" % params[0])
print("r0:   %.3f" % params[1])
print("phi0: %.3f" % params[2])
print("tm:   %.3f" % params[3])
print("Average %% error (estimated to fitted): %.2f" % (err * 100))

# predict future retweets
_, total, pred_error = predict(events=events, obs_time=obs_time, pred_time=pred_time, p_max=None, params=params,
                               **add_params)

print("Predicted number of retweets from %s to %s hours: %i" % (obs_time, pred_time, total))
print("Predicted number of retweets at hour %s: %i" % (pred_time,
                                                       len([e for e, _ in events if e <= obs_time]) + total))
print("Prediction error (absolute): %.0f" % pred_error)

# Estimating the kernel

### sample file

In [ ]:
from tick.hawkes import SimuHawkesSumExpKernels, HawkesSumExpKern, ModelHawkesSumExpKernLeastSq, HawkesExpKern
from tick.plot import plot_point_process, plot_basis_kernels, plot_hawkes_kernels
import itertools
import datetime
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
filename = os.path.join(TIDEH_path, 'data/example/sample_file.txt')
(_, start_time), events = load_events(filename)

In [ ]:
seqs = []
for seq in events:
    seqs.append(seq[0])
seqs = [np.asarray(seqs)]

HawkesSumExpKern <br>
<https://x-datainitiative.github.io/tick/modules/generated/tick.hawkes.HawkesSumExpKern.html?highlight=hawkessumexpkern> <br>
<https://x-datainitiative.github.io/tick/auto_examples/plot_hawkes_varying_baseline.html?highlight=hawkessumexpkern>

In [ ]:
decays = [0.1]
print (datetime.datetime.now())
learner = HawkesSumExpKern(decays, penalty='elasticnet', elastic_net_ratio=0.8)
learner.fit(seqs)
print (datetime.datetime.now())

In [ ]:
flattend = list(itertools.chain.from_iterable(seqs))
t_min = int(np.ceil(np.min(flattend)))
t_max = int(np.floor(np.max(flattend)))

In [ ]:
learner.plot_estimated_intensity(seqs, t_min=t_min, t_max=t_max)

In [ ]:
fig = plot_hawkes_kernels(learner, show=True)

### Multiple training examples

In [ ]:
from tideh import load_events_vec
from tideh import training_cross_validation

number_of_files = 100  # number of files to train on
file_name_prefix = os.path.join(TIDEH_path, 'data/training/RT')  # file names prefix of files used for training
iterations = 5 # number of cross validation iterations
pred_time = 168  # prediction time (hours)

# get file paths of files to use for training
file_names = [file_name_prefix + str(i) + '.txt' for i in range(1, number_of_files + 1)]

# load events for optimized training
events_data = []
for file in file_names:
    events_data.append(load_events_vec(file, 1 / 3600, 24))  # convert event_times and start_time to hours

In [ ]:
seqs = []
for seq in events_data:
    seqs.append(seq[1][0])

HawkesSumExpKern(decays[, penalty, C, …])<br>
Hawkes process learner for sum-exponential kernels with fixed and given decays, with many choices of penalization and solvers.

In [ ]:
decays = [0.1]
print (datetime.datetime.now())
learner = HawkesSumExpKern(decays, penalty='elasticnet', elastic_net_ratio=0.8, n_baselines=2, period_length=100)
learner.fit(seqs)
print (datetime.datetime.now())

In [ ]:
flattend = list(itertools.chain.from_iterable(seqs))
t_min = int(np.ceil(np.min(flattend)))
t_max = int(np.floor(np.max(flattend)))

In [ ]:
print (datetime.datetime.now())
learner.plot_estimated_intensity(seqs, t_min=t_min, t_max=t_max)
print (datetime.datetime.now())

In [ ]:
# fig = plot_hawkes_kernels(learner, show=True)